In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
from unidecode import unidecode

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [2]:
ambiente_origem = "A008"

In [4]:
ambientes_destino_texto = """A109 - SANITÁRIO FEMININO (1º Andar)
A111 - SANITÁRIO MASCULINO (1º Andar)
A209 - SANITÁRIO PDC FEMININO (2º Andar)
A210 - SANITÁRIO PCD FEMININO (2º Andar)
A212 - SANITÁRIO MASCULINO (2º Andar)
A214 - SANITÁRIO MASCULINO (2º Andar)
A308 - SANITÁRIO PCD FEMININO (3º Andar)
A309 - SANITÁRIO PCD FEMININO (3º Andar)
A311 - SANITÁRIO PCD MASCULINO (3º Andar)
A312 - SANITÁRIO MASCULINO (3º Andar)
B006 - SANITÁRIO MASCULINO (Térreo)
B007 - SANITÁRIO FEMININO (Térreo)
B009 - SANITÁRIO MASCULINO [INATIVO] (Térreo)
B101 - SANITÁRIO FEMININO (1º Andar)
B102 - SANITÁRIO MASCULINO (1º Andar)
B108 - SANITÁRIO MASCULINO (1º Andar)
B109 - SANITÁRIO FEMININO (1º Andar)"""

In [33]:
AMBIENTES_DESTINO = [ambiente.split(" - ")[0] for ambiente in ambientes_destino_texto.split("\n")]

In [42]:
pesquisa = db.collection("ambientes-norte") 

BANHEIROS_NORTE = []
for ambiente_norte in pesquisa.stream():
    doc = ambiente_norte.to_dict()
    if doc["detalhes"]["tipo_de_ambiente"] == "Banheiro":
        BANHEIROS_NORTE.append(doc)

In [44]:
len(AMBIENTES_DESTINO)

17

In [45]:
BANHEIROS_NORTE

[{'origem': [],
  'codigo': 'A022',
  'nome': 'Banheiros 01 (Masculino e Feminino)',
  'campus': 'Norte',
  'detalhes': {'observacoes': '',
   'ala': 'Esquerda',
   'setor_responsavel': 'Administração de Edifícios',
   'dimensao_m': 23.47,
   'nomenclatura': 'Banheiro 01',
   'pavimento': 'Térreo',
   'descricao': '',
   'sigla': '',
   'tipo_de_ambiente': 'Banheiro',
   'construcao': 'Bloco A'}},
 {'origem': [],
  'codigo': 'A027',
  'nome': 'Banheiros 02 (Feminino)',
  'campus': 'Norte',
  'detalhes': {'observacoes': '',
   'ala': 'Esquerda',
   'setor_responsavel': 'Administração de Edifícios',
   'dimensao_m': 23.53,
   'nomenclatura': 'Banheiro 02',
   'pavimento': 'Térreo',
   'descricao': '',
   'sigla': '',
   'tipo_de_ambiente': 'Banheiro',
   'construcao': 'Bloco A'}},
 {'origem': [],
  'codigo': 'A028',
  'nome': 'Banheiros 03 (Masculino)',
  'campus': 'Norte',
  'detalhes': {'observacoes': '',
   'ala': 'Esquerda',
   'setor_responsavel': 'Administração de Edifícios',
   'd

In [46]:
def destino(ambiente_origem: str):
    if ambiente_origem.startswith("A"):
        return db.document("ambientes-norte/A022")
    elif ambiente_origem.startswith("B"):
        return db.document("ambientes-norte/B006")
    else:
        return None
    

## Itens de origem

In [11]:
itens_banheiro = []
origem_ref = db.collection('ambientes').document(ambiente_origem)
items_banheiro = db.collection('items').where(filter=FieldFilter('ambiente', '==', origem_ref))
for item in items_banheiro.stream():
    itens_banheiro.append(item.to_dict())

In [60]:
ordem_itens = [
 '1 Espelho de 64 x 44 x 2cm',
 '1 Dispenser de Sabonete Liquido de 8 x 8 x 16cm',
 '1 Dispenser para Álcool de 25 x 11 x 11cm',
 '1 Lixeiro pequeno de 28 x 28 x 33cm',
 '1 Lixeiro de 41 x 31 x 78cm',
 '1 Porta Toalha de 30 x 25 x 12cm',
 '1 Porta Papel Higiênico de 26 x 28 x 14cm'
 ]

AGRUPAMENTO_VOLUMES = [[0], [1, 2, 3], [4], [5, 6]]

MEDIDAS_VOLUMES = [(64, 44, 2), (40, 40, 50), (78, 41, 31), (40, 40, 50)] 

EMBALAGENS_VOLUMES = [None, 'Caixa_M', None, 'Caixa_M']

PROPRIEDADES_VOLUMES = [["Frágil"], [], [], []]

CATEGORIAS_VOLUMES = ["Vidraria", "Outros", "Outros", "Outros"]

def generate_medida_dict(medidas: tuple):
    return {
        "a": medidas[0],
        "c": medidas[1],
        "l": medidas[2]
    }

ITENS_BANHEIRO_ORDENADOS = []

for ordem in ordem_itens:
    for item in itens_banheiro:
        if item['short_descricao'] == ordem:
            ITENS_BANHEIRO_ORDENADOS.append(item)

In [25]:
def generate_itens_para_banheiro(cod_banheiro: str):
    itens_para_banheiro = []
    for (n,item) in enumerate(ITENS_BANHEIRO_ORDENADOS):
        new_item  = item.copy()
        new_item['ambiente'] = db.collection('ambientes').document(cod_banheiro)
        new_item['key'] = f"{cod_banheiro}WC{n:02d}"
        new_item['origem'] = cod_banheiro
        itens_para_banheiro.append(new_item)
    return itens_para_banheiro


# Volume exemplo

In [30]:

VOLUME_EXEMPLO = db.document("volumes/B1230028").get().to_dict()

In [69]:
from datetime import datetime

In [95]:
def create_new_volume(
        codigo: str,
        origem_str: str,
        destino,
        categoria: str,
        items: list,
        medidas: dict,
        embalagem: str | None = None,
        propriedades: list = [],
        ):
    new_volume = VOLUME_EXEMPLO.copy() 
    new_volume['codigo'] = codigo
    new_volume['origem'] = db.document(f"ambientes/{origem_str}")
    new_volume['destino'] = destino
    new_volume['categoria'] = categoria
    new_volume['items'] = items
    new_volume['medidas'] = medidas
    new_volume['embalagem']['caixa'] = embalagem
    new_volume['propriedades'] = propriedades
    new_volume['data_criacao'] = datetime.now()
    new_volume['responsavel'] = db.document(f"usuarios/fabio.metzner@ufsc.br")
    
    return new_volume

In [96]:
def generate_volumes_banheiro(lista_itens: list):
    origem = lista_itens[0]['origem']
    volumes = []
    for (n, grupo) in enumerate(AGRUPAMENTO_VOLUMES):
        volume = create_new_volume(
            codigo = f"{origem}{n:04d}",
            origem_str = origem,
            destino = destino(origem),
            categoria = CATEGORIAS_VOLUMES[n],
            items = [db.document(f"items/{lista_itens[i]['key']}") for i in grupo],
            medidas = generate_medida_dict(MEDIDAS_VOLUMES[n]),
            embalagem = EMBALAGENS_VOLUMES[n],
            propriedades = PROPRIEDADES_VOLUMES[n]
        )
        volumes.append(volume)
    return volumes

# Criando os baguio

In [97]:
primeiros_itens = generate_itens_para_banheiro(ambiente_origem)
primeiros_volumes = generate_volumes_banheiro(primeiros_itens)

ITENS_BANHEIROS_CRIADOS =  primeiros_itens 
VOLUMES_BANHEIRO_CRIADOS = primeiros_volumes 



for banheiro in AMBIENTES_DESTINO:
    itens_banheiro = generate_itens_para_banheiro(banheiro)
    volumes_banheiro = generate_volumes_banheiro(itens_banheiro) 
    ITENS_BANHEIROS_CRIADOS.extend(itens_banheiro)
    VOLUMES_BANHEIRO_CRIADOS.extend(volumes_banheiro)

In [79]:
VOLUMES_BANHEIRO_CRIADOS

[{'medidas': {'a': 64, 'c': 44, 'l': 2},
  'categoria': 'Vidraria',
  'peso': None,
  'items': [<google.cloud.firestore_v1.document.DocumentReference at 0x7fac102fdea0>],
  'observacao': '',
  'servicos': [],
  'status': ['Criado'],
  'data_criacao': datetime.datetime(2023, 12, 6, 16, 45, 28, 562005),
  'embalagem': {'caixa': 'Caixa_M', 'platico_bolha': None, 'enchimento': None},
  'responsavel': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac34368a30>,
  'origem': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac102fe350>,
  'codigo': 'A00800',
  'propriedades': ['Frágil'],
  'destino': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac102ff3a0>,
  'localizacao_atual': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac34213fa0>,
  'deleted': False},
 {'medidas': {'a': 40, 'c': 40, 'l': 50},
  'categoria': 'Outros',
  'peso': None,
  'items': [<google.cloud.firestore_v1.document.DocumentReference at 0x7fac102fea40>,
  'observacao'

# Escrevendo

In [75]:
def batch_write(collection: str, dictarray: list[dict], key_id: str | None = None):
    batch = db.batch()
    collection_ref = db.collection(collection)
    k = 1
    for i in range(0, len(dictarray), 500):
        chunk = dictarray[i:i+500]
        for document in chunk:
            if key_id:
                doc_ref = collection_ref.document(str(document[key_id]))
                batch.set(doc_ref, document)
        print(f"{collection}: chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [86]:
batch_write("items", ITENS_BANHEIROS_CRIADOS, "key")

items: chunk 1 (126 documents)


In [99]:
batch_write("volumes", VOLUMES_BANHEIRO_CRIADOS, "codigo")

volumes: chunk 1 (72 documents)


In [88]:
VOLUMES_BANHEIRO_CRIADOS

[{'medidas': {'a': 64, 'c': 44, 'l': 2},
  'categoria': 'Vidraria',
  'peso': None,
  'items': [<google.cloud.firestore_v1.document.DocumentReference at 0x7fac1037ffd0>],
  'observacao': '',
  'servicos': [],
  'status': ['Criado'],
  'data_criacao': datetime.datetime(2023, 12, 6, 16, 48, 1, 917944),
  'embalagem': {'caixa': 'Caixa_M', 'platico_bolha': None, 'enchimento': None},
  'responsavel': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac1037c310>,
  'origem': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac1037dd50>,
  'codigo': 'A0080000',
  'propriedades': ['Frágil'],
  'destino': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac1037e590>,
  'localizacao_atual': <google.cloud.firestore_v1.document.DocumentReference at 0x7fac34213fa0>,
  'deleted': False},
 {'medidas': {'a': 40, 'c': 40, 'l': 50},
  'categoria': 'Outros',
  'peso': None,
  'items': [<google.cloud.firestore_v1.document.DocumentReference at 0x7fac1037d630>,
  'observacao

Consertando cagadas 

```python
origens_refs = [db.document(f"ambientes/{x}") for x in [ambiente_origem ] + ambientes_destino]

pesquisa = db.collection("volumes").where(filter=FieldFilter("origem", 'in', origens_refs))
for volume in pesquisa.stream():
    volume.reference.delete()

```
